<a href="https://colab.research.google.com/github/viniciusgino06/codigos/blob/main/Monitora_Para.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=sjTsP3BtEMAQHUuOO8Or_1phyt91-1hnyAsALwt3Pjo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5qEZVqIxlZBCiMMqnGqjkfJg2Vt4xwxMb4v4CUVfPchsx9xwHbJtk

Successfully saved authorization token.


In [2]:
import pandas as pd
import altair as alt
import numpy as np
import folium

In [3]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=30,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):

  def reduce_region_function(img):
    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function

In [4]:
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

In [5]:
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

In [46]:
#Altamira: "-53.087722265293074,-3.0827040401895953,-52.546645605136824,-2.6630060677497744"
#Belo Monte: "-51.77791578129848,-3.1746053346429606,-51.45381910161098,-2.9277635680293748"
#BR : "-53.601333105136824,-3.5090901157025236,-53.30058237271495,-3.296605429036365"
#Alto Rio Xingu: "-46.85906054491093,-2.3557968672031806,-46.81511523241093,-2.3261242333120435"
#Altamira_2 : "-52.21376191028843,-3.761735468180357,-52.06201325306187,-3.6692331710058648"

coordenadas = "-46.85906054491093,-2.3557968672031806,-46.81511523241093,-2.3261242333120435"
x1,y1,x2,y2 = coordenadas.split(",")
geometria = geometry = ee.Geometry.Polygon(
        [[[float(x1),float(y2)],
          [float(x2),float(y2)],
          [float(x2),float(y1)],
          [float(x1),float(y1)],
          [float(x1),float(y2)]]])

#ndvi = ee.ImageCollection("LANDSAT/LC08/C01/T1_8DAY_NDVI").select("NDVI")
colecao = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(geometria)
blue = 'B2'; green = 'B3'; red = 'B4'; nir = 'B5'; pan = 'B8'

def ndvi(imagem):
    ndvi = imagem.expression('(nir - red) / (nir + red)',{'nir':imagem.select(nir),'red':imagem.select(red)}).rename('ndvi')
    return imagem.addBands(ndvi)

cole = colecao.map(ndvi)
ndvi = cole.select('ndvi')
reduce_ndvi = create_reduce_region_function(
    geometry=geometria, reducer=ee.Reducer.mean(), scale=30)

ndvi_stat_fc = ee.FeatureCollection(ndvi.map(reduce_ndvi)).filter(
    ee.Filter.notNull(ndvi.first().bandNames()))

In [41]:
ndvi2 = cole.select('ndvi')

In [10]:
print("LANDSAT 8: "+str(cole.size().getInfo()))

LANDSAT 8: 119


In [47]:
ndvi_dict = fc_to_dict(ndvi_stat_fc).getInfo()
ndvi_df = pd.DataFrame(ndvi_dict)
display(ndvi_df)
print(ndvi_df.dtypes)

,millis,ndvi,system:index
0,1367414346340,0.416113,LC08_222062_20130501
1,1368796758740,0.571886,LC08_222062_20130517
2,1371561555380,0.562222,LC08_222062_20130618
3,1372943957817,0.304992,LC08_222062_20130704
4,1375708759580,0.541085,LC08_222062_20130805
...,...,...,...
114,1607951859790,0.465191,LC08_222062_20201214
115,1609334256271,0.398056,LC08_222062_20201230
116,1614863836306,0.635277,LC08_222062_20210304
117,1616246227860,0.403829,LC08_222062_20210320


millis            int64
ndvi            float64
system:index     object
dtype: object


In [48]:
ndvi_df = add_date_info(ndvi_df)
ndvi_df.head(5)

,millis,ndvi,system:index,Timestamp,Year,Month,Day,DOY
0,1367414346340,0.416113,LC08_222062_20130501,2013-05-01 13:19:06.340,2013,5,1,121
1,1368796758740,0.571886,LC08_222062_20130517,2013-05-17 13:19:18.740,2013,5,17,137
2,1371561555380,0.562222,LC08_222062_20130618,2013-06-18 13:19:15.380,2013,6,18,169
3,1372943957817,0.304992,LC08_222062_20130704,2013-07-04 13:19:17.817,2013,7,4,185
4,1375708759580,0.541085,LC08_222062_20130805,2013-08-05 13:19:19.580,2013,8,5,217


In [49]:
highlight = alt.selection(
    type='single', on='mouseover', fields=['Year'], nearest=True)

base = alt.Chart(ndvi_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 353], clamp=True)),
    y=alt.Y('ndvi:Q', scale=alt.Scale(domain=[0, 1])),
    color=alt.Color('Year:O', scale=alt.Scale(scheme='magma')))

points = base.mark_circle().encode(
    opacity=alt.value(0),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('DOY:Q', title='DOY'),
        alt.Tooltip('ndvi:Q', title='NDVI')
    ]).add_selection(highlight)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3)))

(points + lines).properties(width=600, height=350).interactive()

alt.LayerChart(...)

In [52]:
base = alt.Chart(ndvi_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=(0, 360))))

line = base.mark_line().encode(
    y=alt.Y('median(ndvi):Q', scale=alt.Scale(domain=(0, 1))))

band = base.mark_errorband(extent='iqr').encode(
    y='ndvi:Q')

(line + band).properties(width=600, height=300).interactive()

alt.LayerChart(...)

In [53]:
#Altamira_2: "-52.88942108997593,-3.087972339514653,-52.26869355091343,-2.7011991135450955"
#Altamira_1: "-51.77791578129848,-3.1746053346429606,-51.45381910161098,-2.9277635680293748"
#parakana: "-50.4821822409366,-4.811656769546225,-50.28339836642488,-4.691564664330002"
coordenadas = "-50.4821822409366,-4.811656769546225,-50.28339836642488,-4.691564664330002"
x1,y1,x2,y2 = coordenadas.split(",")
geometria = geometry = ee.Geometry.Polygon(
        [[[float(x1),float(y2)],
          [float(x2),float(y2)],
          [float(x2),float(y1)],
          [float(x1),float(y1)],
          [float(x1),float(y2)]]])

ndvi = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')

reduce_ndvi = create_reduce_region_function(
    geometry=geometria, reducer=ee.Reducer.mean(), scale=1000)

ndvi_stat_fc = ee.FeatureCollection(ndvi.map(reduce_ndvi)).filter(
    ee.Filter.notNull(ndvi.first().bandNames()))

In [54]:
print("MODIS: "+str(ndvi.size().getInfo()))

MODIS: 488


In [55]:
ndvi_dict = fc_to_dict(ndvi_stat_fc).getInfo()
ndvi_df = pd.DataFrame(ndvi_dict)
display(ndvi_df)
print(ndvi_df.dtypes)

,NDVI,millis,system:index
0,2220.394730,950832000000,2000_02_18
1,5116.572264,952214400000,2000_03_05
2,5966.632432,953596800000,2000_03_21
3,6108.810284,954979200000,2000_04_06
4,7465.255266,956361600000,2000_04_22
...,...,...,...
483,6422.084227,1613606400000,2021_02_18
484,2671.857766,1614988800000,2021_03_06
485,7901.860841,1616371200000,2021_03_22
486,7888.832010,1617753600000,2021_04_07


NDVI            float64
millis            int64
system:index     object
dtype: object


In [56]:
ndvi_df['NDVI'] = ndvi_df['NDVI'] / 10000
ndvi_df = add_date_info(ndvi_df)
ndvi_df.head(5)

,NDVI,millis,system:index,Timestamp,Year,Month,Day,DOY
0,0.222039,950832000000,2000_02_18,2000-02-18,2000,2,18,49
1,0.511657,952214400000,2000_03_05,2000-03-05,2000,3,5,65
2,0.596663,953596800000,2000_03_21,2000-03-21,2000,3,21,81
3,0.610881,954979200000,2000_04_06,2000-04-06,2000,4,6,97
4,0.746526,956361600000,2000_04_22,2000-04-22,2000,4,22,113


In [57]:
highlight = alt.selection(
    type='single', on='mouseover', fields=['Year'], nearest=True)

base = alt.Chart(ndvi_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 353], clamp=True)),
    y=alt.Y('NDVI:Q', scale=alt.Scale(domain=[0, 1])),
    color=alt.Color('Year:O', scale=alt.Scale(scheme='magma')))

points = base.mark_circle().encode(
    opacity=alt.value(0),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('DOY:Q', title='DOY'),
        alt.Tooltip('NDVI:Q', title='NDVI')
    ]).add_selection(highlight)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3)))

(points + lines).properties(width=600, height=350).interactive()

alt.LayerChart(...)

In [58]:
base = alt.Chart(ndvi_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=(0, 360))))

line = base.mark_line().encode(
    y=alt.Y('median(NDVI):Q', scale=alt.Scale(domain=(0, 1))))

band = base.mark_errorband(extent='iqr').encode(
    y='NDVI:Q')

(line + band).properties(width=600, height=300).interactive()

alt.LayerChart(...)

In [42]:
lonc = (float(x1)+float(x2))/2
latc = (float(y1)+float(y2))/2
my_map = folium.Map(location=[latc, lonc], zoom_start=10)

In [39]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [43]:
parametros = {
  'min': 1030.398741410466,
  'max': 10000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}
params = {
  'min': 0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

img = ndvi.median()
img2 = ndvi2.median()
my_map.add_ee_layer(img,parametros,'NDVI_MODIS')
my_map.add_ee_layer(img2,params,'NDVI_L8')
my_map.add_child(folium.LayerControl())
display(my_map)